In [1]:
# Parámetro para generar el modelo del taggeador
data = False
%store data

Stored 'data' (bool)


In [2]:
%%time
%run ./carga_diputados.ipynb

Wall time: 2.28 s


In [3]:
%%time
%run ./carga_dialogos.ipynb

False


Documentos.....: 100%|██████████| 10/10 [43:57<00:00, 263.76s/it] 

Wall time: 47min 50s


In [4]:
import neo4j_connector as nc
import pandas as pd

In [5]:
graph = nc.generate_graph()

In [6]:
grupos_parlamentarios = list(graph.run('MATCH (d:Diputado) RETURN DISTINCT(d.grupo) AS grupo ORDER BY grupo ASC').to_data_frame()['grupo'])

### OBTENEMOS LISTA DE RELACIONES

In [7]:
query = 'MATCH ()-[relation]-() RETURN DISTINCT(relation.pleno) as sesion order by sesion asc'
sesiones = graph.run(query).to_data_frame()

In [8]:
def lista_palabras(grupo, plenos):
    query = "MATCH (d:Diputado)-[rel]-(p:Palabra) "
    query = query + "WHERE d.grupo = '" + grupo +"' "
    query = query + "AND rel.pleno IN " + str(plenos) + " "
    query = query + "RETURN rel.pleno, d.grupo, p.palabra, SUM(rel.veces) as veces "
    query = query + "ORDER BY veces DESC LIMIT 15"
    return graph.run(query).to_data_frame()

In [27]:
historico = []
for ix, sesion in sesiones.iterrows():
    historico.append(sesion['sesion'])
    print("SESION...... " + sesion['sesion'])
    for gr in grupos_parlamentarios:
        print(gr)
        palabras_mas_dichas = lista_palabras(gr, [sesion['sesion']])
        palabras_mas_dichas_hist = lista_palabras(gr, historico)       
        print(palabras_mas_dichas)
        print(palabras_mas_dichas_hist)

    break


SESION...... 00_INVESTIDURA
Grupo Ciudadanos
         rel.pleno           d.grupo   p.palabra  veces
0   00_INVESTIDURA  Grupo Ciudadanos     sánchez     79
1   00_INVESTIDURA  Grupo Ciudadanos     acuerdo     58
2   00_INVESTIDURA  Grupo Ciudadanos      españa     55
3   00_INVESTIDURA  Grupo Ciudadanos    gobierno     45
4   00_INVESTIDURA  Grupo Ciudadanos   españoles     39
5   00_INVESTIDURA  Grupo Ciudadanos     partido     38
6   00_INVESTIDURA  Grupo Ciudadanos         día     31
7   00_INVESTIDURA  Grupo Ciudadanos    esquerra     30
8   00_INVESTIDURA  Grupo Ciudadanos    cataluña     28
9   00_INVESTIDURA  Grupo Ciudadanos        país     20
10  00_INVESTIDURA  Grupo Ciudadanos  socialista     19
11  00_INVESTIDURA  Grupo Ciudadanos       dicho     18
12  00_INVESTIDURA  Grupo Ciudadanos     podemos     18
13  00_INVESTIDURA  Grupo Ciudadanos         pnv     18
14  00_INVESTIDURA  Grupo Ciudadanos       decía     18
         rel.pleno           d.grupo   p.palabra  veces
0  

In [18]:
for gupo in grupos_parlamentarios:
    print(grupo)

Grupo Socialista
Grupo Socialista
Grupo Socialista
Grupo Socialista
Grupo Socialista
Grupo Socialista
Grupo Socialista
Grupo Socialista
Grupo Socialista
Grupo Socialista


In [25]:
type(grupos_parlamentarios)

list